# Projekt 4: Constraint-Satisfaction Probleme
**Abgabe:** 19.07.2024

Wir werden zur Modellierung der CSPs die Bibliothek `PyCSP3` (https://pycsp.org/) benutzen. Macht euch vor der Bearbeitung mit diese Bibliothek vertraut. Beachtet, dass diese Bibliothek intern einen Java-Solver benutzt. Daher müsst ihr Java auf eurem Computer installiert haben.

In [8]:
from pycsp3 import *

Hier ist eine Beispielmodellierung des n-Damen Problems. Beachtet, dass das Aufrufen von `clear()` nach der `solve()` Methode notwendig ist, um den Solver zurückzusetzen. In manchen Fällen gibt es trotzdem Fehlermeldungen. Dann hilft ein Neustart des Python-Kernels.

In [ ]:
n = 100

x = VarArray(size=n, dom=range(n))

satisfy(
    AllDifferent(x),
    AllDifferent(x[i] - i for  i in range(n)),
    AllDifferent(x[i] + i for i in range(n))
)

if solve() is SAT:
    print(values(x))
clear()

## 1. Modellierung

In dieser Aufgabe sollen verschiedene kombinatorische Probleme modelliert werden. Für jedes Problem stehen im Ordner `test` einige Testfälle bereit. Modellieren Sie die Probleme mithilfe der `PyCSP3`-Bibliothek und evaluieren Sie anschließend die Modellierung anhand der test-Instanzen.

#### a) Killer-Sudoku

Ein Killer-Sudoku ist ein Standard-Sudoku mit zusätzlichen Summen-Constraints. Zusätzlich zu den üblichen Regeln, dass jede Zeile, Spalte und die 3x3 Felder jeweils die Zahlen von 1-9 enthalten müssen und in keiner Reihe, Spalte, Box eine Zahl doppelt vorkommen darf, gibt es zusätzliche Summen-Constraints. Dabei muss die Summe in den umrandeten Feldern genau der angegeben Summe entsprechen. Im Folgenden ist ein Beispiel Killer-Sudoku zu sehen:
<center>
<img src="killer_sudoku.png" width="400" height="400">
</center>


In [ ]:
def solve_killer_sudoku(cages):
    pass

#### b) Trees and Tents
In dieser Aufgabe soll das Spiel "Trees and Tents" (https://www.brainbashers.com/showtents.asp) modelliert werden. Das Spiel folgt ein paar sehr einfachen Regeln. Gespielt wird auf einem $n\times n$-Feld. Am Anfang stehen auf dem Feld Bäume. Die Aufgabe ist es nun Zelte auf diesem Feld nach folgenden Regeln zu platzieren:
- Jedes Zelt gehört zu genau einem Baum und zu jedem Baum gehört genau ein Zelt
- Die Anzahl der Zelte entspricht der Anzahl der Bäume
- Die Zahlen geben an, wie viele Zelt in der jeweiligen Spalte und Zeile stehen
- Ein Zelt gehört nur zu einem Baum, wenn es horizontal oder vertikal mit diesem verbunden ist.
- Zelte dürfen sich niemals berühren (weder horizontal, vertikal oder diagonal)

<center>
<img src="trees.png" width="300">
</center>

In [ ]:
def solve_trees(board, row_sums, col_sums):
    pass
    

#### c) Pattern Encodings
Ziel ist es die Felder in einem Raster schwarz zu malen, sodass Blöcke aufeinanderfolgender schwarzer Felder die für jede Reihe und Spalte gegebenen Einschränkungen erfüllen. Einschränkungen geben die Abfolge von schwarzen Blöcken an (z.B. bedeutet 3,1,2, dass es einen Block von 3 Feldern gibt, dann eine Lücke von unbestimmter Größe, einen Block von 1 Feld, eine weitere Lücke und schließlich einen Block von 2 Feldern). Unten befindet sich ein Beispiel
<center>
<img src="pattern_solved.png" width="300">
</center>

**Hinweis**: Modelliert das Problem ohne die Benutzung des `regular`-Constraints! Schreiben sie außerdem eine Funktion um das Bild eines solchen Patterns darzustellen.

In [ ]:
heart_example_rows = [[2, 2], [4, 4], [1, 3, 1], [2, 1, 2], [1, 1], [2, 2], [2, 2], [3], [1]]
heart_example_cols = [[3], [2, 3], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 3], [3]]

In [ ]:
def solve_pattern_encoding(rows, cols):
    pass


In [ ]:
def visualize_pattern(pattern_solution):
    pass

## 2. CSP-Solving
In dieser Aufgabe soll ein einfacher CSP-Solver implementiert werden. Dafür finden Sie unten die teilweise vorimplementierte Klasse `CSP`. Wir betrachten dabei nur binäre Constraints der Form $(x,y, f_\text{check})$ mit $x,y\in \text{Vars}$, für die gilt der Constraint ist erfüllt genau dann wenn $f_\text{check}(\texttt{assignment}[x], \texttt{assignment}[y])=\texttt{True}$. Die Klasse CSP verwendet zur Darstellung zwei Dictionaries. Das dict `domains`, weist jeder Variables eine Menge von möglichen Werten zu. Das dict `constraints`, weißt Tupeln von Variablen eine Menge von `check`-Funktionen zu, die für dieses Variablenpaar gelten müssen.

Implementiere nun die restlichen Funktionen zum Lösen solcher CSP. Zum einen sollte ein einfacher Backtracking-Algorithmus zum Lösen implementiert werden. Anschließend soll der AC-3 Algorithmus aus der Vorlesung zur Herstellung der Arc-Consistency implementiert werden. Implementieren Sie außerdem die Minimum Remaining Values (MRV) Heuristik zum Auswählen der nächsten Variable. Bei dieser Heuristik wird diejenige Variable als nächstes belegt, welche die kleinste verbliebene Domain hat.

Evaluiert Eure Implementierung anschließend sowohl an dem Testbeispiel, als auch an den verschiedenen Sudokus.

In [1]:
import copy

In [8]:
class CSP:
    """
    Solves csp instances with ac3 and backtracking.
    Only works if given constraints contains boths arcs.
    """
    def __init__(self, domains: dict, constraints: dict):
        self.domains = domains
        self.constraints = constraints

    
    def complete(self, n):
        """ Checks if csp is solved """
        if None in n.values():
            return False
            
        for c in self.constraints:
            for f in self.constraints[c]:
                
                if not f(n[c[0]], n[c[1]]):
                    return False
                    
        return True

    
    def consistent(self, n, x, v):
        """ Checks if n[x -> v] is consistent """
        n[x] = v
        
        for c in self.constraints:
            
            if x not in c or n[c[0]] is None or n[c[1]] is None:
                continue
                
            for f in self.constraints[c]:
                
                if not f(n[c[0]], n[c[1]]):
                    n[x] = None
                    return False
                    
        return True

    
    def simple_backtracking(self, n=None, ac=False, counter=0): # del counter
        """ Solves csp via backtracking """
        if ac and n is None:
            self.ac3()
            
        n = dict.fromkeys(self.domains.keys()) if n is None else n
        if self.complete(n):
            return n, counter
        x = self.choose(n)
        
        for v in self.domains[x]:
            if self.consistent(n, x, v):
                
                if ac:
                    # Use ac3 to propagate constraints to domains
                    copy_domains = copy.deepcopy(self.domains)
                    self.domains[x] = {v}
                    inf = self.ac3(x=x)
                else:
                    inf = True
                    
                if inf:
                    counter += 1
                    r = self.simple_backtracking(n=n, ac=ac, counter=counter)
                    if r:
                        return r
                     
            n[x] = None
            self.domains = copy_domains if ac else self.domains
        return False
                    
        
    def choose(self, n):
        """ Chosse variable via Minimum Remaining Values heuristic """
        it = iter(n.keys())
        res = next(it)
        for k in it:
            res = k if n[k] is None and len(self.domains[k]) <= len(self.domains[res]) else res
        return res
            
        
    def ac3(self, x=None) -> dict:
        """ arc consistency 3 """
        queue = [c for c in self.constraints if x is None or c[0] == x or c[1] == x]
        
        while queue:
            c = queue.pop(0)
            if self.revise(c):
                if not self.domains[c[0]] or not self.domains[c[1]]:
                    return False
                    
                for n in self.constraints:
                    if n not in queue and n[1] == c[0] and n[0] != c[1]:
                        queue.append(n)
                        
        return self.domains, self.constraints

    
    def revise(self, constraint):
        res = False
        copy = self.domains[constraint[0]].copy()
        
        for i in copy:
            satisfies = False
            for j in self.domains[constraint[1]]:
                for f in self.constraints[constraint]:
                    if f(i, j):
                        satisfies = True
                        break
                        
            if not satisfies:
                self.domains[constraint[0]].remove(i)
                res = True
                
        return res
        

    def ac3_backtrack(self):
        return self.simple_backtracking(ac=True)


In [10]:
domains = {
    'a': {2, 3, 4, 5, 6, 7},
    'b': {4, 5, 6, 7, 8, 9},
    'c': {1, 2, 3, 4, 5}
}

# constraints:
# b = 2*a
# a = c
# b >= c - 2
# b <= c + 2

constraints = {
    ('a', 'b'): {lambda a, b: a * 2 == b},
    ('b', 'a'): {lambda b, a: b == 2 * a},
    ('a', 'c'): {lambda a, c: a == c},
    ('c', 'a'): {lambda c, a: c == a},
    ('b', 'c'): {lambda b, c: b >= c - 2, lambda b, c: b <= c + 2},
    ('c', 'b'): {lambda c, b: b >= c - 2, lambda c, b: b <= c + 2},
}

"""
Die Lösung der Instanz sollte {'a': 2, 'b': 4, 'c': 2} sein.
"""

csp = CSP(domains, constraints)
print(csp.simple_backtracking())
print(csp.ac3_backtrack())


({'a': 2, 'b': 4, 'c': 2}, 4)
({'a': 2, 'b': 4, 'c': 2}, 3)


In [11]:
# Sudoku instances (index i belongs to row i // 9 and column i % 9)
easy = [
    '.9......45.2.......13......95.3.16.2.267.54...3..6..7...4....131.....5.7.6.......',
    '...97..81.2......9.4..38.5.5..1...............9......72....73..3.98......8..15.4.',
    '..8.....9.3...7.4857..........9.....6.4.......5....836.9....567.61.5.98....3..1..'
]
medium = [
    '.275....85....6..9.......2..91.27...7...632.........713...9...5.........1.9....63',
    '.38.6.71..9...............3......5746..1...3..8.5.76..54..96..8.1.7..4...........',
    '895...6.2...8.4..7......5..............681...56.4.71..9..72....1..9..72...8......'
]
hard = [
    '.3.9.67.565.7.24......5.3............7.3.....4....8.2.8....7....9..2.......4.9.8.',
    '3.7...5.9...7..46..9.4.38...5.379......1....8..9.....64......1...8.........614...',
    '.......98...7....38.9...51..31.2........78......3....661.....39.......6...2.918..'
]

def build_csp(instance):
    domains, constraints = {}, {}
    for i, n in enumerate(instance):
        x, y = i // 9, i % 9
        domains[str(x)+str(y)] = {_ for _ in range(9)} if n == "." else {int(n)}

        for j in range(9):
            if j != x:
                constraints[str(x)+str(y), str(j)+str(y)] = {lambda a, b: a != b}
            if j != y:
                constraints[str(x)+str(y), str(x)+str(j)] = {lambda a, b: a != b}

        block_x = x // 3 * 3
        block_y = y // 3 * 3
        for k in range(block_x, block_x + 3):
            for l in range(block_y, block_y + 3):
                if (x, y) != (k, l):
                    constraints[str(x)+str(y), str(k)+str(l)] = {lambda a, b: a != b}
        
    return domains, constraints

# evaluate
sodokus = [build_csp(i) for i in easy+medium+hard]
"""
for s in sodokus:
    csp = CSP(*s)
    # TODO timer
    csp.simple_backtracking()
    csp.ac3_backtrack()
"""

csp = CSP(*sodokus[1])
print(csp.simple_backtracking())
print(csp.ac3_backtrack())

({'00': 6, '01': 3, '02': 5, '03': 9, '04': 7, '05': 4, '06': 0, '07': 8, '08': 1, '10': 8, '11': 2, '12': 0, '13': 6, '14': 5, '15': 1, '16': 4, '17': 7, '18': 9, '20': 1, '21': 4, '22': 7, '23': 0, '24': 3, '25': 8, '26': 6, '27': 5, '28': 2, '30': 5, '31': 7, '32': 2, '33': 1, '34': 4, '35': 0, '36': 8, '37': 3, '38': 6, '40': 0, '41': 1, '42': 3, '43': 7, '44': 8, '45': 6, '46': 5, '47': 2, '48': 4, '50': 4, '51': 9, '52': 8, '53': 5, '54': 2, '55': 3, '56': 1, '57': 0, '58': 7, '60': 2, '61': 5, '62': 1, '63': 4, '64': 0, '65': 7, '66': 3, '67': 6, '68': 8, '70': 3, '71': 0, '72': 9, '73': 8, '74': 6, '75': 2, '76': 7, '77': 1, '78': 5, '80': 7, '81': 8, '82': 6, '83': 3, '84': 1, '85': 5, '86': 2, '87': 4, '88': 0}, 95)


RecursionError: maximum recursion depth exceeded while calling a Python object